# Comparison performances DBMS / Ontology

## PostgreSQL

In [1]:
%%time
from models import Airport, Frequency

Wall time: 992 ms


In [2]:
def test_request_postgre():
    s, n, w, e = 42.69998105639557, 44.49946827276767, 0.20684058124504337, 2.6917189331881017

    # Query Airports
    near_airports = Airport.query.filter( \
                    (Airport.longitude >= w) & (Airport.longitude <= e) & \
                    (Airport.latitude >= s) & (Airport.latitude <= n)) \
                        .with_entities(Airport.name, Airport.iata, Airport.icao, Airport.latitude, Airport.longitude, Airport.altitude, Airport.country)\
                        .all()
    dict_airports_dbms = [r._asdict() for r in near_airports]

    # Query Frequencies
    for airport in dict_airports_dbms:
        current_icao = airport['icao']
        associated_frequencies = Frequency.query.filter(Frequency.airport == current_icao)\
                                    .with_entities(Frequency.frq_type, Frequency.desc, Frequency.frq_mhz)\
                                    .all()                              
        airport['list_frequencies'] = [r._asdict() for r in associated_frequencies]
    
    return dict_airports_dbms

In [3]:
dict_airports_dbms = test_request_postgre()
dict_airports_dbms

[{'name': 'Agen La Garenne Airport',
  'iata': 'AGF',
  'icao': 'LFBA',
  'latitude': 44.174709,
  'longitude': 0.590619,
  'altitude': 204.0,
  'country': 'France',
  'list_frequencies': [{'frq_type': 'APP',
    'desc': 'BLAGNAC APP',
    'frq_mhz': 121.1},
   {'frq_type': 'APP', 'desc': 'TOULOUSE APP', 'frq_mhz': 121.175},
   {'frq_type': 'ATIS', 'desc': 'ATIS', 'frq_mhz': 129.6},
   {'frq_type': 'TWR', 'desc': 'AGEN TWR', 'frq_mhz': 121.3}]},
 {'name': 'Albi Le Sequestre Airport',
  'iata': 'LBI',
  'icao': 'LFCI',
  'latitude': 43.913269,
  'longitude': 2.11675,
  'altitude': 564.0,
  'country': 'France',
  'list_frequencies': [{'frq_type': 'AFIS',
    'desc': 'ALBI INFO',
    'frq_mhz': 118.95},
   {'frq_type': 'APP', 'desc': 'TOULOUSE APP', 'frq_mhz': 123.85}]},
 {'name': 'Carcassonne Airport',
  'iata': 'CCF',
  'icao': 'LFMK',
  'latitude': 43.215809,
  'longitude': 2.308544,
  'altitude': 433.0,
  'country': 'France',
  'list_frequencies': [{'frq_type': 'APP',
    'desc': 'TOU

## Ontology

In [4]:
%%time
import owlready2 as owl
filename_onto_individuals = "../../src/ontology/final-archi-individuals.owl"
onto_individuals = owl.get_ontology(filename_onto_individuals).load()

Wall time: 6.93 s


In [5]:
def test_ontolgy():
    # Query Airports
    s, n, w, e = 42.69998105639557, 44.49946827276767, 0.20684058124504337, 2.6917189331881017

    near_airports = list(owl.default_world.sparql(
        f"""
            PREFIX pie:<http://www.semanticweb.org/clement/ontologies/2020/1/final-archi#>
            SELECT ?name ?iata ?icao ?latitude ?longitude ?altitude ?country
            WHERE {{
                ?Airport pie:AirportName ?name .
                ?Airport pie:AirportIATA ?iata .
                ?Airport pie:AirportICAOCode ?icao .
                ?Airport pie:AirportGPSLatitude ?latitude .
                ?Airport pie:AirportGPSLongitude ?longitude .
                ?Airport pie:AirportAltitude ?altitude .
                ?Airport pie:AirportCountry ?country .
                FILTER (?longitude > {w} && ?longitude < {e} 
                    &&  ?latitude > {s} && ?latitude < {n})

            }}
        """))
    fields = ['name', 'iata', 'icao', 'latitude', 'longitude', 'altitude', 'country']
    dict_airports_onto = [dict(zip(fields, airport_tuple)) for airport_tuple in near_airports]

    # Query Frequencies
    for airport in dict_airports_onto:
        current_icao = airport['icao']
        associated_frequencies = list(owl.default_world.sparql(
            f"""
                PREFIX pie:<http://www.semanticweb.org/clement/ontologies/2020/1/final-archi#>
                SELECT ?frq_type ?desc ?frq_mhz 
                WHERE {{
                    ?Airport pie:AirportICAOCode ?ICAO .
                    ?Airport pie:HasFrequency ?Frequency .
                    ?Frequency pie:FrequencyDescription ?desc .
                    ?Frequency pie:FrequencyMHz ?frq_mhz .
                    ?Frequency pie:FrequencyType ?frq_type .
                    FILTER regex(?ICAO, "{current_icao}", "i")

                }}
            """))
        fields = ['frq_type', 'desc', 'frq_mhz']
        airport['list_frequencies'] = [dict(zip(fields, frq_tuple)) for frq_tuple in associated_frequencies]
    
    return dict_airports_onto

In [6]:
dict_airports_onto = test_ontolgy()
dict_airports_onto

[{'name': 'Carcassonne Airport',
  'iata': 'CCF',
  'icao': 'LFMK',
  'latitude': 43.215809,
  'longitude': 2.308544,
  'altitude': 433,
  'country': 'France',
  'list_frequencies': [{'frq_type': 'APP',
    'desc': 'TOULOUSE APP',
    'frq_mhz': 123.85},
   {'frq_type': 'ATIS', 'desc': 'ATIS', 'frq_mhz': 120.025},
   {'frq_type': 'TWR', 'desc': 'CARCASSONNE TWR', 'frq_mhz': 121.0}]},
 {'name': 'Castres Mazamet Airport',
  'iata': 'DCM',
  'icao': 'LFCK',
  'latitude': 43.554958,
  'longitude': 2.290592,
  'altitude': 787,
  'country': 'France',
  'list_frequencies': [{'frq_type': 'AFIS',
    'desc': 'CASTRES INFO',
    'frq_mhz': 118.5},
   {'frq_type': 'APP', 'desc': 'TOULOUSE APP', 'frq_mhz': 123.85}]},
 {'name': 'Toulouse Blagnac Airport',
  'iata': 'TLS',
  'icao': 'LFBO',
  'latitude': 43.634998,
  'longitude': 1.367778,
  'altitude': 499,
  'country': 'France',
  'list_frequencies': [{'frq_type': 'AFIS',
    'desc': 'TOULOUSE INFO',
    'frq_mhz': 121.25},
   {'frq_type': 'APP', 

## Compare performances

In [7]:
onto_list = sorted(dict_airports_onto, key=lambda x: x['name'])
dbms_list = sorted(dict_airports_dbms, key=lambda x: x['name'])
onto_list == dbms_list

True

In [8]:
%%timeit
test_request_postgre()

18 ms ± 2.46 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
%%timeit
test_ontolgy()

1.29 s ± 162 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
